# Dekoratory funkcji i klas

## Domknięcie (*closure*)

Domknięcie, w metodach realizacji języków programowania, jest to obiekt **wiążący funkcję oraz środowisko**, w jakim ta funkcja ma działać.
Środowisko przechowuje wszystkie obiekty wykorzystywane przez funkcję, niedostępne w globalnym zakresie widoczności.
Realizacja domknięcia jest zdeterminowana przez język, jak również przez kompilator.

Domknięcia występują głównie w językach funkcyjnych, w których funkcje mogą zwracać inne funkcje, wykorzystujące zmienne utworzone lokalnie.

In [ ]:
def bind_add(x):
    def add(y):
        # x jest "zamknięte" w definicji
        return y + x
    return add

In [ ]:
add_5 = bind_add(5)
add_5(10)

15

In [ ]:
add_665 = bind_add(665)
add_665(2)

667

## Wprowadzenie do dekoratorów

Dekorator to wzorzec projektowy, pozwalający na dynamiczne dodanie nowej funkcjonalności, w trakcie działania programu.

W języku Python jest to metoda modyfikacji obiektu wywoływalnego (funkcji, metod klasy, klas) za pomocą domknięć.

Dekoratory są w Pythonie często spotykaną techniką programistyczną.
Ich zalety to redukcja ilości kodu oraz możliwość kontrolowania funkcji (lub innych obiektów wywoływalnych), w szczególności ich danych wejściowych i zwracanych wartości.

## Prosty dekorator

Poniżej przedstawiono implementację dekoratora `@shouter`.
Funkcje udekorowane nim wyświetlają komunikat na początku i pod koniec ich wywołania.


In [ ]:
def shouter(func):
    def wrapper():
        print("Before", func.__name__)
        result = func()
        print(result)
        print("After", func.__name__)
        return result
    return wrapper

Można tak zdefiniowanej funkcji użyć do "nadpisania" istniejącej już funkcji (tak naprawdę do zmiany tego, na co wskazuje zmienna):

In [ ]:
def greetings():
    return "Hi"

hello = shouter(greetings)

hello()

Before greetings
Hi
After greetings


'Hi'

Począwszy od Pythona 2.4, możliwe i rekomendowane jest użycie specjalnej składni:

In [ ]:
@shouter
def hello():
    return "Hello"

In [ ]:
hello()

Before hello
Hello
After hello


'Hello'

Użycie ``@shouter`` przed definicją funkcji jest równoważne umieszczeniu za jej definicją linii ``hello = shouter(hello)``.

## Argumenty w dekoratorach

Problem
*******

Przedstawiony dekorator działa tylko z funkcjami, które nie przyjmują żadnych argumentów.
Co z funkcjami wymagającymi argumentów?

In [ ]:
@shouter
def add(x, y):
    '''Docstring for add(x, y)'''
    return x + y

In [ ]:
add(2, 7)

TypeError: wrapper() takes 0 positional arguments but 2 were given

Innym problemem jest to, że udekorowana funkcja utraciła swój docstring oraz swoją nazwę:

In [ ]:
add.__doc__

In [ ]:
add.__name__

'wrapper'

Rozwiązanie
***********

Argumenty przekazywane do *wrapper* muszą zostać przekazane dalej, do właściwej funkcji *func*.

Z kolei problem z docstringiem i nazwą rozwiążemy dekorując funkcję *wrapper* przy pomocy dekoratora ``@functools.wraps``, który zadba o skopiowanie docstringa i nazwy:


In [ ]:
import functools

def shouter(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print("Before", func.__name__)
        result = func(*args, **kwargs)
        print(result)
        print("After", func.__name__)
        return result
    return wrapper

In [ ]:
@shouter
def add(x, y):
    '''Docstring for add(x, y)'''
    return x + y

In [ ]:
add(5, 6)

Before add
11
After add


11

In [ ]:

add.__doc__

'Docstring for add(x, y)'

In [ ]:
add.__name__

'add'

## Dekoratory parametryzowane

Dekoratory, które nie przyjmują żadnych argumentów, są często spotykane.
Jednak czasami potrzebujemy przekazać do dekoratora argumenty. 

Aby otrzymać parametryzowany dekorator, musimy go "owinąć" w jeszcze jedną funkcję (domknięcie):

In [ ]:
def tag(tagname):
    def decorator(fun):
        @functools.wraps(fun)
        def wrapper(*args, **kwargs):
            tag_before = f"<{tagname}>"
            tag_after = f"</{tagname}>"
            fresult= fun(*args, **kwargs)            
            return tag_before + fresult + tag_after
        return wrapper
    return decorator

In [ ]:
@tag("b")
def output(data):
    return data

In [ ]:
output("TEXT")

'<b>TEXT</b>'

Użycie ``@tag("b")`` jest odpowiednikiem:

In [ ]:
output = tag("b")(output)

## Wiele dekoratorów

Funkcję można owijać w wiele dekoratorów.

In [ ]:
@shouter
@tag('b')
def my_func(text):
    return text

In [ ]:
my_func("text")

Before my_func
<b>text</b>
After my_func


'<b>text</b>'

co odpowiada:

In [ ]:
my_func = shouter(tag("b")(my_func))

Należy pamiętać, że kolejność ma znaczenie.

## Kiedy uruchamiane są dekoratory

Kluczowe znaczenie dla dekoratorów ma fakt, że są one uruchamiane zaraz po tym jak zdefiniowana została dekorowana funkcja. 
Najczęściej jest to moment *importu* pakietu.

In [ ]:
registry = []  

def register(func):
    print(f'running register({func})') 
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')

def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

if __name__ == '__main__':
    main()

running register(<function f1 at 0x7f4677b749d0>)
running register(<function f2 at 0x7f4677b74790>)
running main()
registry -> [<function f1 at 0x7f4677b749d0>, <function f2 at 0x7f4677b74790>]
running f1()
running f2()
running f3()


## Dekoratory klas

Od Pythona 2.6 można dekorować klasy.
W środku dekoratora można zmodyfikować klasę, na przykład zmienić jej metody.
Dekoratory klas mają działanie zbliżone do metaklas.


In [ ]:
id = 0

def add_id(decorated_class):
    original_init = decorated_class.__init__
    
    def __init__(self, *args, **kwargs):
        print("add_id init")
        global id
        id += 1
        self.id = id
        original_init(self, *args, **kwargs)
    
    decorated_class.__init__ = __init__ # replacing __init__ in decorated class
    return decorated_class

@add_id
class Foo(object):
    def __init__(self):
        print("Foo class init")


In [ ]:
foo = Foo()
foo.id

add_id init
Foo class init


1

In [ ]:
bar = Foo()
bar.id

add_id init
Foo class init


2

## Klasy jako dekoratory

Bardzo ciekawym zastosowaniem jest użycie klasy jako dekoratora.
Wystarczy zdefiniować w klasie metodę specjalną `__call__`.
Instancja klasy (uzyskana za pomocą operatora ``()``) staje się wtedy obiektem, który można wywołać.

Jest to alternatywa dla definiowania nieparametryzowanego dekoratora przy pomocy dwóch zagnieżdżonych funkcji.
Kod jest nieco prostszy do zrozumienia:

In [ ]:
class shout:
    def __init__(self, f):
        print("inside decorator's __init__()")
        self.f = f
        
    def __call__(self):
        print("before call")
        self.f()
        print("after call")

In [ ]:
@shout
def function():
    print("inside function()")

inside decorator's __init__()


In [ ]:
function()

before call
inside function()
after call
